In [ ]:
import torch
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils
import cv2

## data preprocess

In [ ]:
ava = pd.read_csv('ava_alltxt.csv')
print('Length of ava_train : {}' .format(len(ava)))

In [ ]:
ava['filename'] = ava['ImageIndex'].apply(lambda x: str(x)+'.jpg')

#### get a dataframe with all file names

In [ ]:
df_all = pd.DataFrame({
    'filename': ava['filename']
})

print('Shape of df: {}'.format(df_all.shape))
print('Preview of df: ')
print(df_all[:3])

####  read data by file name order and resize data to 224*224*3

In [ ]:
path = 'all_avatxt/'
X_all = []
def create_test_data(path):        
    for filename in df_all['filename']: #assuming 
    #     read RGB files and resize picture to 224*224*3

        img = cv2.imread(os.path.join(path,filename),1)       
        resized_im = cv2.resize(img, (224,224), interpolation = cv2.INTER_AREA)

        X_all.append(resized_im)

        
create_test_data(path)
X_all = np.array(X_all)
Y_all = ava['score']

In [ ]:
train_images = X_all[:23087]
train_labels  = Y_all[:23087]

In [ ]:
train_y = []
for i in train_label:
    if i<=5:
        train_y.append(0)
    else:
        train_y.append(1)
train_y = np.array(train_y)

In [ ]:
from keras.utils import to_categorical
train_label = to_categorical(train_y)

In [ ]:
# save intermidate data
# np.save('all_data_224.npy', X_all)

## build model

In [ ]:
FAST_RUN = False
IMAGE_WIDTH=128
IMAGE_HEIGHT=128
IMAGE_SIZE=(IMAGE_WIDTH, IMAGE_HEIGHT)
IMAGE_CHANNELS=3

In [ ]:
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Dropout, Flatten, Dense, Activation, BatchNormalization

In [ ]:
model = Sequential()

model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(IMAGE_WIDTH, IMAGE_HEIGHT, IMAGE_CHANNELS)))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(512, activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.5))
model.add(Dense(2, activation='softmax')) # 2 because we have cat and dog classes

model.compile(loss='categorical_crossentropy', optimizer='rmsprop', metrics=['accuracy'])

model.summary()

In [ ]:
history = model.fit(train_images, train_label, epochs=150, batch_size=32, validation_split=0.2)

##  model result

In [ ]:
fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(12, 12))
ax1.plot(history.history['loss'], color='b', label="Training loss")
ax1.plot(history.history['val_loss'], color='r', label="validation loss")
ax1.set_xticks(np.arange(1, 150, 15))
ax1.set_yticks(np.arange(0, 2, 0.1))

ax2.plot(history.history['acc'], color='b', label="Training accuracy")
ax2.plot(history.history['val_acc'], color='r',label="Validation accuracy")
ax2.set_xticks(np.arange(1, 150, 15))

legend = plt.legend(loc='best', shadow=True)
plt.tight_layout()
plt.show()

## test

In [ ]:
X_test = np.array(X_all[23087:])
y_test = np.array(Y_all[23087:])


In [ ]:
predictions = model.predict(X_test)

In [ ]:
df_result = pd.DataFrame({ 'label':predicted_val,'y':y_test})
df_result['y_test'] = df_result['label'].apply(lambda x: 1 if x>=5 else 0)
df_result.head()

In [ ]:
df_result['que'] = np.where((df_result['y_test'] == df_result['y'])
                     , 1, 0)

In [ ]:
df_result['que'].value_counts()